# Model3_multi_collinear

The purpose of this notebook / model experiment is to improve upon the previous fix_imbalance model achieved from model2_fix_imbalance.ipynb, by removing multicollinearity

In [1]:
import pandas as pd
from pycaret.classification import *

## import data

In [2]:
df_ts_agg = pd.read_csv("../proData/df_ts_agg.csv")
df_ts_agg = df_ts_agg.set_index("PATIENT_ID")

df_ts_agg

ihd  Age  Gender  Height                        ICUType  \
PATIENT_ID                                                            
132539        0   54  female     NaN                   Surgical ICU   
132540        0   76    male   175.3  Cardiac Surgery Recovery Unit   
132541        0   44  female     NaN                    Medical ICU   
132543        0   68    male   180.3                    Medical ICU   
132545        0   88  female     NaN                    Medical ICU   
...         ...  ...     ...     ...                            ...   
142665        0   70  female     NaN                   Surgical ICU   
142667        0   25    male     NaN                    Medical ICU   
142670        0   44    male     NaN                    Medical ICU   
142671        1   37    male     NaN                    Medical ICU   
142673        0   78  female   157.5                   Surgical ICU   

            Weight_min  Weight_max  Weight_med  GCS_min  GCS_max  ...  \
PATIENT_ID                                                        ...   
132539             NaN         NaN         NaN     14.0     15.0  ...   
132540            76.0        81.6        80.6      3.0     15.0  ...   
132541            56.7        56.7        56.7      5.0      8.0  ...   
132543            84.6        84.6        84.6     14.0     15.0  ...   
132545             NaN         NaN         NaN     15.0     15.0  ...   
...                ...         ...         ...      ...      ...  ...   
142665            87.0        87.0        87.0      3.0     15.0  ...   
142667           166.4       166.4       166.4     15.0     15.0  ...   
142670           109.0       109.0       109.0      3.0      8.0  ...   
142671            87.4        87.4        87.4      3.0      7.0  ...   
142673            70.7        88.8        88.8      3.0     15.0  ...   

            Lactate_med  Cholesterol_min  Cholesterol_max  Cholesterol_med  \
PATIENT_ID                                                                   
132539              NaN              NaN              NaN              NaN   
132540              NaN              NaN              NaN              NaN   
132541              1.3              NaN              NaN              NaN   
132543              NaN              NaN              NaN              NaN   
132545              NaN              NaN              NaN              NaN   
...                 ...              ...              ...              ...   
142665              2.3              NaN              NaN              NaN   
142667              NaN            117.0            117.0            117.0   
142670              NaN              NaN              NaN              NaN   
142671              1.9              NaN              NaN              NaN   
142673              2.6              NaN              NaN              NaN   

            TroponinI_min  TroponinI_max  TroponinI_med  TroponinT_min  \
PATIENT_ID                                                               
132539                NaN            NaN            NaN            NaN   
132540                NaN            NaN            NaN            NaN   
132541                NaN            NaN            NaN            NaN   
132543                NaN            NaN            NaN            NaN   
132545                NaN            NaN            NaN            NaN   
...                   ...            ...            ...            ...   
142665                NaN            NaN            NaN            NaN   
142667                NaN            NaN            NaN            NaN   
142670                NaN            NaN            NaN            NaN   
142671                NaN            NaN            NaN            NaN   
142673                NaN            NaN            NaN            NaN   

            TroponinT_max  TroponinT_med  
PATIENT_ID                                
132539                NaN            NaN  
132540                NaN      

## setup experiment with pycaret

with "remove_multicollinearity" set as true, some linear kernal-based model performance is expected to be improved

In [3]:
exp_physionet = setup(data = df_ts_agg, target = 'ihd', session_id=123,
                      numeric_features = ["GCS_min", "GCS_max"],
                      #normalize = True, 
                      #transformation = True, 
                      #ignore_low_variance = True,
                      remove_multicollinearity = True, multicollinearity_threshold = 0.9,
                      fix_imbalance = True
                     ) 

Setup Succesfully Completed!


## compare common classification models

min(Recall, Precision) is used for comparing, as suggested by the original physionet 2012 challenge

In [4]:
compare_models(turbo = False)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

by removing multicollinearity, some linear kernel-based classifier is increased

## create, tune, evaluate, predict gradient boosting classifier

In [11]:
gbc = create_model("gbc")

In [17]:
tuned_gbc = tune_model(gbc, optimize = "F1", choose_better = True)

In [18]:
evaluate_model(tuned_gbc)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [19]:
predict_model(tuned_gbc)

Model  Accuracy     AUC  Recall   Prec.      F1  \
0  Gradient Boosting Classifier     0.856  0.8217  0.3072  0.4679  0.3709   

    Kappa     MCC  
0  0.2935  0.3018

Age      Height  Weight_max  GCS_min  GCS_max  GCS_med  HR_min  HR_max  \
0     74.0  160.000000   103.00000      3.0     15.0     11.5    79.0   100.0   
1     57.0  169.677895    68.00000     14.0     15.0     15.0    89.0   161.0   
2     90.0  170.200000    69.00000     11.0     15.0     14.0    50.0   113.0   
3     43.0  169.677895    84.04358     15.0     15.0     15.0    77.0   112.0   
4     66.0  169.677895    45.00000      9.0     11.0     10.0    50.0    81.0   
...    ...         ...         ...      ...      ...      ...     ...     ...   
1196  81.0  157.500000    67.50000     10.0     14.0     12.0    69.0   104.0   
1197  58.0  169.677895    71.60000     14.0     15.0     15.0    57.0   126.0   
1198  73.0  169.677895    66.00000     14.0     15.0     15.0   104.0   139.0   
1199  19.0  169.677895    90.00000      7.0     14.0      8.0    83.0   121.0   
1200  82.0  152.400000    74.20000      3.0     15.0     11.0    80.0    93.0   

      HR_med  NIDiasABP_min  ...  ICUType_Cardiac Surgery Recovery Unit  \
0       88.0      30.000000  ...                                    1.0   
1      109.0      31.000000  ...                                    0.0   
2       78.5      40.000000  ...                                    0.0   
3       93.5      49.000000  ...                                    0.0   
4       62.0      49.000000  ...                                    1.0   
...      ...            ...  ...                                    ...   
1196    81.0      35.000000  ...                                    0.0   
1197    77.0      40.000000  ...                                    1.0   
1198   123.0      33.000000  ...                                    0.0   
1199   103.0      55.000000  ...                                    0.0   
1200    81.0      40.814474  ...                                    1.0   

      ICUType_Coronary Care Unit  ICUType_Medical ICU  ICUType_Surgical ICU  \
0                            0.0                  0.0                   0.0   
1                            0.0                  1.0                   0.0   
2                            0.0                  0.0                   1.0   
3                            0.0                  0.0                   1.0   
4                            0.0                  0.0                   0.0   
...                          ...                  ...                   ...   
1196                         0.0                  0.0                   1.0   
1197                         0.0                  0.0                   0.0   
1198                         0.0                  1.0                   0.0   
1199                         0.0                  0.0                   1.0   
1200                         0.0                  0.0                   0.0   

      MechVent_min_0  MechVent_max_1  MechVent_med_0.5  ihd  Label   Score  
0                1.0             1.0               0.0    0      0  0.0657  
1                1.0             0.0               0.0    0      0  0.2504  
2                1.0             0.0               0.0    1      0  0.2276  
3                1.0             0.0               0.0    0      0  0.0164  
4                1.0             1.0               0.0    0      0  0.0945  
...              ...             ...               ...  ...    ...     ...  
1196             1.0             0.0               0.0    1      0  0.4819  
1197             1.0             0.0               0.0    0      0  0.0299  
1198             1.0             0.0               0.0    1      0  0.1429  
1199             1.0             1.0               0.0    0      0  0.1690  
1200             1.0             1.0               0.0    0      0  0.0720  

[1201 rows x 93 columns]

## explore blend models, both hard and soft voting

In [4]:
blend_hard = blend_models()

In [5]:
evaluate_model(blend_hard)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [6]:
predict_model(blend_hard)

Model  Accuracy  AUC  Recall   Prec.     F1   Kappa     MCC
0  Voting Classifier    0.8451    0  0.3193  0.4206  0.363  0.2767  0.2801

Age      Height  Weight_max  GCS_min  GCS_max  GCS_med  HR_min  HR_max  \
0     74.0  160.000000   103.00000      3.0     15.0     11.5    79.0   100.0   
1     57.0  169.677895    68.00000     14.0     15.0     15.0    89.0   161.0   
2     90.0  170.200000    69.00000     11.0     15.0     14.0    50.0   113.0   
3     43.0  169.677895    84.04358     15.0     15.0     15.0    77.0   112.0   
4     66.0  169.677895    45.00000      9.0     11.0     10.0    50.0    81.0   
...    ...         ...         ...      ...      ...      ...     ...     ...   
1196  81.0  157.500000    67.50000     10.0     14.0     12.0    69.0   104.0   
1197  58.0  169.677895    71.60000     14.0     15.0     15.0    57.0   126.0   
1198  73.0  169.677895    66.00000     14.0     15.0     15.0   104.0   139.0   
1199  19.0  169.677895    90.00000      7.0     14.0      8.0    83.0   121.0   
1200  82.0  152.400000    74.20000      3.0     15.0     11.0    80.0    93.0   

      HR_med  NIDiasABP_min  ...  Gender_not_available  \
0       88.0      30.000000  ...                   0.0   
1      109.0      31.000000  ...                   0.0   
2       78.5      40.000000  ...                   0.0   
3       93.5      49.000000  ...                   0.0   
4       62.0      49.000000  ...                   0.0   
...      ...            ...  ...                   ...   
1196    81.0      35.000000  ...                   0.0   
1197    77.0      40.000000  ...                   0.0   
1198   123.0      33.000000  ...                   0.0   
1199   103.0      55.000000  ...                   0.0   
1200    81.0      40.814474  ...                   0.0   

      ICUType_Cardiac Surgery Recovery Unit  ICUType_Coronary Care Unit  \
0                                       1.0                         0.0   
1                                       0.0                         0.0   
2                                       0.0                         0.0   
3                                       0.0                         0.0   
4                                       1.0                         0.0   
...                                     ...                         ...   
1196                                    0.0                         0.0   
1197                                    1.0                         0.0   
1198                                    0.0                         0.0   
1199                                    0.0                         0.0   
1200                                    1.0                         0.0   

      ICUType_Medical ICU  ICUType_Surgical ICU  MechVent_min_0  \
0                     0.0                   0.0             1.0   
1                     1.0                   0.0             1.0   
2                     0.0                   1.0             1.0   
3                     0.0                   1.0             1.0   
4                     0.0                   0.0             1.0   
...                   ...                   ...             ...   
1196                  0.0                   1.0             1.0   
1197                  0.0                   0.0             1.0   
1198                  1.0                   0.0             1.0   
1199                  0.0                   1.0             1.0   
1200                  0.0                   0.0             1.0   

      MechVent_max_1  MechVent_med_0.5  ihd  Label  
0                1.0               0.0    0      0  
1                0.0               0.0    0      0  
2                0.0               0.0    1      0  
3                0.0               0.0    0      0  
4                1.0               0.0    0      0  
...              ...               ...  ...    ...  
1196             0.0               0.0    1      0  
1197             0.0               0.0    0      0  
1198             0.0               0.0    1      0  
1199             1.0               0.0    0      0  
1200             1.0               0.0    0     

In [7]:
blend_soft = blend_models(method = "soft")

In [8]:
evaluate_model(blend_soft)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [9]:
predict_model(blend_soft)

Model  Accuracy     AUC  Recall   Prec.      F1  Kappa     MCC
0  Voting Classifier    0.8443  0.8171  0.4217  0.4348  0.4281  0.338  0.3381

Age      Height  Weight_max  GCS_min  GCS_max  GCS_med  HR_min  HR_max  \
0     74.0  160.000000   103.00000      3.0     15.0     11.5    79.0   100.0   
1     57.0  169.677895    68.00000     14.0     15.0     15.0    89.0   161.0   
2     90.0  170.200000    69.00000     11.0     15.0     14.0    50.0   113.0   
3     43.0  169.677895    84.04358     15.0     15.0     15.0    77.0   112.0   
4     66.0  169.677895    45.00000      9.0     11.0     10.0    50.0    81.0   
...    ...         ...         ...      ...      ...      ...     ...     ...   
1196  81.0  157.500000    67.50000     10.0     14.0     12.0    69.0   104.0   
1197  58.0  169.677895    71.60000     14.0     15.0     15.0    57.0   126.0   
1198  73.0  169.677895    66.00000     14.0     15.0     15.0   104.0   139.0   
1199  19.0  169.677895    90.00000      7.0     14.0      8.0    83.0   121.0   
1200  82.0  152.400000    74.20000      3.0     15.0     11.0    80.0    93.0   

      HR_med  NIDiasABP_min  ...  ICUType_Cardiac Surgery Recovery Unit  \
0       88.0      30.000000  ...                                    1.0   
1      109.0      31.000000  ...                                    0.0   
2       78.5      40.000000  ...                                    0.0   
3       93.5      49.000000  ...                                    0.0   
4       62.0      49.000000  ...                                    1.0   
...      ...            ...  ...                                    ...   
1196    81.0      35.000000  ...                                    0.0   
1197    77.0      40.000000  ...                                    1.0   
1198   123.0      33.000000  ...                                    0.0   
1199   103.0      55.000000  ...                                    0.0   
1200    81.0      40.814474  ...                                    1.0   

      ICUType_Coronary Care Unit  ICUType_Medical ICU  ICUType_Surgical ICU  \
0                            0.0                  0.0                   0.0   
1                            0.0                  1.0                   0.0   
2                            0.0                  0.0                   1.0   
3                            0.0                  0.0                   1.0   
4                            0.0                  0.0                   0.0   
...                          ...                  ...                   ...   
1196                         0.0                  0.0                   1.0   
1197                         0.0                  0.0                   0.0   
1198                         0.0                  1.0                   0.0   
1199                         0.0                  0.0                   1.0   
1200                         0.0                  0.0                   0.0   

      MechVent_min_0  MechVent_max_1  MechVent_med_0.5  ihd  Label   Score  
0                1.0             1.0               0.0    0      0  0.0804  
1                1.0             0.0               0.0    0      0  0.3977  
2                1.0             0.0               0.0    1      1  0.5006  
3                1.0             0.0               0.0    0      0  0.0926  
4                1.0             1.0               0.0    0      0  0.3470  
...              ...             ...               ...  ...    ...     ...  
1196             1.0             0.0               0.0    1      1  0.5070  
1197             1.0             0.0               0.0    0      0  0.1018  
1198             1.0             0.0               0.0    1      0  0.2953  
1199             1.0             1.0               0.0    0      0  0.1935  
1200             1.0             1.0               0.0    0      0  0.2199  

[1201 rows x 93 columns]

## conclusion

building upon the imbalance-fixed model, through additional removal of multicollinearity, either a single model, e.g. gradient boosting classifier, or blend models has increased train or test min(recall, precision) to 0.43 or 0.42